**Ex 1:**

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.1 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
coef = np.loadtxt('lab4ex1_coef.txt', delimiter = ',')

In [5]:
print(coef.shape)
print('Number of rows=', coef.shape[0])
print('Number of columns=', coef.shape[1])

(6, 7)
Number of rows= 6
Number of columns= 7


In [6]:
print(coef)

[[  0.85   3.75  -8.75  -3.45 -22.38  20.     0.  ]
 [ -0.7   -3.9   -1.     0.     1.     0.    14.9 ]
 [  1.     0.     0.    -1.     0.    -1.     0.9 ]
 [  0.    -3.    -2.     0.     0.    -1.   -51.7 ]
 [  0.     0.     0.     4.8    9.12   7.2   75.  ]
 [ -0.8   -1.5    8.    -2.     1.     0.   -27.  ]]


In [7]:

model = ConcreteModel()

In [8]:
M = coef.shape[0] - 1 
N = coef.shape[1] - 1 

In [9]:
print(M)
print(N)

5
6


In [10]:
#coefficients of the objective function
obj_coef = coef[0,:-1]


In [11]:
#coefficients of the constraints
constr_coef = coef[1:,:-1]


In [12]:
#rhs
constr_rhs = coef[1:,-1]


In [13]:
l_bound = 0
u_bound = np.inf

In [14]:
row_indices = np.arange(M)
col_indices = np.arange(N)

In [15]:
print(col_indices)
print(row_indices)

[0 1 2 3 4 5]
[0 1 2 3 4]


In [16]:
# declare the decision variables in the model
model.x = Var(col_indices)

In [17]:
model.constraints = ConstraintList()

In [18]:
for i in row_indices:
      model.constraints.add(sum(constr_coef[i][j]*model.x[j] for j in col_indices)<= constr_rhs[i])

In [20]:
# setting upper and lower bounds
for j in col_indices:
  model.x[j].setlb(l_bound)
  model.x[j].setub(u_bound)

In [21]:
model.objective = Objective(expr = summation(obj_coef, model.x), sense = maximize)

In [22]:
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [23]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148486 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [24]:
opt_cbc = SolverFactory('cbc')

In [25]:
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

    model.name="unknown";
      - termination condition: unbounded
      - message from solver: <undefined>
Solver status: warning
Solver termination condition: unbounded


**Ex 1 part 7**

The termination condition is unbounded as in this problem we can maximize the objective function indefinitely while satisfying the constraints.

The solver status shows that we cannot find an optimal solution, as the feasible region is unbounded.

We were solving the problem in the wrong sense, so the feasible region is unbounded and we are getting unbounded solution.

In [26]:
# resetting the sense to minimize
model.objective.set_sense(minimize)
model.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

1 Var Declarations
    x : Size=6, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :   inf : False :  True :  Reals
          1 :     0 :  None :   inf : False :  True :  Reals
          2 :     0 :  None :   inf : False :  True :  Reals
          3 :     0 :  None :   inf : False :  True :  Reals
          4 :     0 :  None :   inf : False :  True :  Reals
          5 :     0 :  None :   inf : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.85*x[0] + 3.75*x[1] - 8.75*x[2

In [27]:
result = opt_cbc.solve(model)
print(result)
print('SolverStatus:', result.solver.status)
print('Termination Condition:', result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: -97.02184211
  Upper bound: -97.02184211
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 7
  Number of nonzeros: 6
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.022484540939331055
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

SolverStatus: ok
Termination Condition: optimal


In [28]:
# display results
print('Objective Function:', model.objective())
print('\nDecision Variables: ')
for j in col_indices:
      print('x[',j,']= ', model.x[j].value)
print('\nConstraints:')
model.constraints.display()      

Objective Function: -97.02184239599997

Decision Variables: 
x[ 0 ]=  0.9
x[ 1 ]=  23.002456
x[ 2 ]=  0.0
x[ 3 ]=  0.0
x[ 4 ]=  8.2236842
x[ 5 ]=  0.0

Constraints:
constraints : Size=5
    Key : Lower : Body                : Upper
      1 :  None :         -82.1158942 :  14.9
      2 :  None :                 0.9 :   0.9
      3 :  None :          -69.007368 : -51.7
      4 :  None :   74.99999990399999 :  75.0
      5 :  None : -26.999999799999998 : -27.0


PART 9)

After changing the sense from maximize to minimize, we are now getting an optimal solution. The previous termination condition(unbounded) was due to wrong sense.

Optimal Objective Value= -97.022

optimal solution is: x1 = 0.9 ; x2 = 23.0024 ; x3 = 0 ; x4 = 0; x5 = 8.2236842 ; x6 = 0

Constraint 2, 4 and 5 are active since they attain equality.